# nodejs 랑 node.js해결해야함

#### getSkillsSet() - skills 리스트 빈도 필터링 없이 다 확인

In [4]:
import csv
import re

In [10]:
# skills 데이터 다 확인

def getSkillsSet():
    with open("../data/crawled_data_combined.csv", encoding="utf-8") as f:
        reader = csv.reader(f)

        # 태그 하나씩 쪼개서 넣을 리스트 (set으로 중복 제거 전)
        skillList = []

        # csv 파일 리더로, 한 row씩 읽기
        for row in reader:

            # row list의 10번째 element인 tags string을 ''안 단어들로 나눠서 list만들기
            skills = re.findall(r"'(.*?)'", row[10])     
            skills_upper = [word.upper() for word in skills]

            # 각 줄의 태그를 tagList에 넣는다.
            skillList += skills_upper
            # print(skills)

        skillsSet = set(skillList)
        
        return skillsSet

####  ListToDict(lst) - skills List로 딕셔너리 만들기

In [11]:
def ListToDict(lst):
    toDict = dict()

    for item in lst:
        toDict[item.upper()] = 0
    
    return toDict
    


#### saveCSV(dict, title) - dictionary key, value를 csv 파일로 저장하는 함수

In [41]:
import csv

def saveCSV(dict, title):
    # Specify the CSV file path

    # Open the CSV file in write mode
    with open(title, "w", newline="") as file:
        writer = csv.writer(file)

        # Write the header row with column names
        writer.writerow(["Key", "Value"])

        # Write each key-value pair as a row in the CSV file
        for key, value in dict.items():
            writer.writerow([key, value])

    print("Dictionary keys and values saved to CSV file successfully.")

In [12]:
comSet = getSkillsSet()
comList = list(comSet)
comDict = ListToDict(comList)

## 각 채용링크의 qualification 속 skill set 정리하기

In [45]:
import re
from tqdm import tqdm
import csv
from konlpy.tag import Komoran

def qualification():
    skillDict = comDict.copy()
    freq_dict = dict()

    with open("../data/crawled_data_combined.csv", 'r', encoding="utf-8") as f:
        reader = csv.reader(f)
        data = list(reader)
        count = 1
        for row in tqdm(data[1:]):
            id = row[0]
            data = row[7].upper()

            # 확인해야 하는 데이터 (list)
            phrases = comList 

            # remove symbols
            sentence = re.sub(r'[^\w\s]', '', data)

            # re.escape으로 특수문자 벗어나고, |로 단어 연결
            pattern = r'\b(' + '|'.join(map(re.escape, phrases)) + r')\b'

            # 한글 정렬후 나누기
            komoran = Komoran()
            sentence = komoran.morphs(sentence)

            # 한글 정렬 후 나누고 다시 문장으로 합쳐주기
            sentence = " ".join(sentence)
            
            # phrases에 있는 단어 sentence속 빈도 확인하기
            matches = re.findall(pattern, sentence)
            

            if count % 500 == 0:
                print(matches)
            count += 1

            # dictionary에 하나씩 넣어주기 - set으로 중복 제거
            freq_dict[id] = list(set(matches))

    saveCSV(freq_dict, "freq_qual_20230703.csv")
    return freq_dict
            

# qualifications = 7, preferred = 8, skills = 10

In [46]:
qualification()

  9%|▉         | 500/5451 [16:46<3:07:02,  2.27s/it]

['SQL', '관리']


 18%|█▊        | 1000/5451 [32:12<2:18:21,  1.87s/it]

['서버', '개발', 'API', '설계', '개발', 'SOLR', 'ELASTICSEARCH', '검색 엔진']


 28%|██▊       | 1500/5451 [47:27<1:52:01,  1.70s/it]

...

 92%|█████████▏| 5000/5451 [2:30:29<10:54,  1.45s/it]

['개발']


100%|██████████| 5451/5451 [2:42:38<00:00,  1.79s/it]

Dictionary keys and values saved to CSV file successfully.


{'120493': ['JAVA', 'SPRING BOOT'],
 '82530': ['AWS', 'MONGODB', '인프라'],
 '158261': ['개발',
  '보안',
  'KOTLIN',
  'RESTFUL API',
  'DOCKER',
  '설계',
  'JAVA',
  'AWS',
  'JPA',
  'RDBMS',
  'NOSQL',
  'SPRING FRAMEWORK',
  'KUBERNETES'],
 '147597': ['하드웨어'],
 '118168': ['개발', 'SPRING BOOT', '서버', 'API', '설계', 'REST'],
 '163951': ['API',
  '컴퓨터 공학',
  'RESTFUL API',
  'HTML',
  'GIT',
  '웹 개발',
  'JAVASCRIPT',
  '관리',
  'CSS',
  '웹 디자인'],
 '151109': ['개발', '컴퓨터 공학', '수학', 'GIS', '최적화'],
 '122553': ['개발'],
 '103598': ['MONGODB',
  'GITLAB',
  'LARAVEL',
  'MYSQL',
  'JQUERY',
  'JAVASCRIPT',
  'PHP'],
 '157234': ['개발',
  'API',
  'DJANGO',
  'HTML',
  'GIT',
  '데이터베이스',
  '클라이언트 개발',
  'PYTHON',
  'REST',
  'JQUERY',
  'JAVASCRIPT',
  'CSS',
  'GITHUB'],
 '164438': ['HTML', 'JAVA', 'MSSQL', 'PYTHON', 'ASP', 'JAVASCRIPT', 'CSS'],
 '119599': ['설계', 'GCP', '서버'],
 '159693': ['정보 보안', '컴퓨터 공학', '보안', '설계', '관리'],
 '72388': ['JIRA',
  '개발',
  'TYPESCRIPT',
  'HTML',
  'GIT',
  'NOTION',
  'RED

In [47]:
import re
from tqdm import tqdm
import csv
from konlpy.tag import Komoran

def preferred():
    skillDict = comDict.copy()
    freq_dict = dict()

    with open("../data/crawled_data_combined.csv", 'r', encoding="utf-8") as f:
        reader = csv.reader(f)
        data = list(reader)
        count = 1
        for row in tqdm(data[1:]):
            id = row[0]
            data = row[8].upper()

            # 확인해야 하는 데이터 (list)
            phrases = comList 

            # remove symbols
            sentence = re.sub(r'[^\w\s]', '', data)

            # re.escape으로 특수문자 벗어나고, |로 단어 연결
            pattern = r'\b(' + '|'.join(map(re.escape, phrases)) + r')\b'

            # 한글 정렬후 나누기
            komoran = Komoran()
            sentence = komoran.morphs(sentence)

            # 한글 정렬 후 나누고 다시 문장으로 합쳐주기
            sentence = " ".join(sentence)
            
            # phrases에 있는 단어 sentence속 빈도 확인하기
            matches = re.findall(pattern, sentence)
            

            if count % 500 == 0:
                print(matches)
            count += 1

            # dictionary에 하나씩 넣어주기 - set으로 중복 제거
            freq_dict[id] = list(set(matches))

    saveCSV(freq_dict, "freq_pref_20230703.csv")
    return freq_dict
            

# qualifications = 7, preferred = 8, skills = 10

In [48]:
preferred()

  9%|▉         | 500/5451 [14:19<2:18:25,  1.68s/it]

['TOMCAT', 'ORACLE', 'MYSQL', 'MSSQL', 'JAVA', '개발']


 18%|█▊        | 1000/5451 [28:44<1:53:21,  1.53s/it]

['KUBERNETES', '개발']


 28%|██▊       | 1500/5451 [43:05<1:55:58,  1.76s/it]

...

 92%|█████████▏| 5000/5451 [2:23:54<13:24,  1.78s/it]

['3D', '개발', 'WEBGL', '3D', '개발']


100%|██████████| 5451/5451 [2:36:58<00:00,  1.73s/it]

Dictionary keys and values saved to CSV file successfully.


{'120493': ['GIT', '관리'],
 '82530': ['설계', 'PYTHON', 'SHELL', '모델링', 'JAVASCRIPT'],
 '158261': ['개발', 'RABBITMQ', 'REDIS'],
 '147597': ['개발'],
 '118168': ['JIRA',
  '개발',
  'API',
  'GRAPHQL',
  'DOCKER',
  '설계',
  'GIT',
  'GITLAB',
  'GOLANG',
  '가상화',
  'KUBERNETES',
  '관리',
  'GITHUB'],
 '163951': ['개발', '보안', 'TYPESCRIPT', '금융', '최적화', '관리'],
 '151109': ['UBUNTU',
  'C',
  'GIT',
  '자동차',
  'VISUAL STUDIO',
  'PYTHON',
  'JENKINS',
  'JIRA',
  'UNREAL ENGINE',
  '로봇'],
 '122553': ['GIT', '개발', '관리', 'GIS'],
 '103598': ['개발'],
 '157234': ['개발', 'GCP', 'AWS', 'ANGULAR'],
 '164438': [],
 '119599': ['리팩토링'],
 '159693': ['개발', '보안', '보안 교육'],
 '72388': ['SEO', '개발'],
 '120367': ['POSTGRESQL', 'GRAPHQL', 'AWS', 'PYTHON', 'FASTAPI', 'FLUTTER'],
 '31024': ['개발', 'B2B', '데이터 분석', 'SQL', 'SAAS'],
 '162844': [],
 '106376': ['설계', '데이터 수집'],
 '37335': ['개발', 'JAVA', 'AWS', 'DEVOPS'],
 '158891': ['개발'],
 '150305': ['개발'],
 '145306': ['SQL', '개발', '데이터베이스', '설계'],
 '146089': ['개발', 'FLOW', '리팩토

## 각 채용링크의 Preferred 속 skill set 정리하기

In [ ]:
import re
from tqdm import tqdm
import csv
from konlpy.tag import Komoran

def preferred():
    skillDict = comDict.copy()
    freq_dict = dict()

    with open("../data/crawled_data_combined.csv", 'r', encoding="utf-8") as f:
        reader = csv.reader(f)
        data = list(reader)
        count = 1
        for row in tqdm(data[1:]):
            id = row[0]
            data = row[8].upper()

            # 확인해야 하는 데이터 (list)
            phrases = comList 

            # remove symbols
            sentence = re.sub(r'[^\w\s]', '', data)

            # re.escape으로 특수문자 벗어나고, |로 단어 연결
            pattern = r'\b(' + '|'.join(map(re.escape, phrases)) + r')\b'

            # 한글 정렬후 나누기
            komoran = Komoran()
            sentence = komoran.morphs(sentence)

            # 한글 정렬 후 나누고 다시 문장으로 합쳐주기
            sentence = " ".join(sentence)
            
            # phrases에 있는 단어 sentence속 빈도 확인하기
            matches = re.findall(pattern, sentence)
            

            if count % 500 == 0:
                print(matches)
            count += 1

            # dictionary에 하나씩 넣어주기 - set으로 중복 제거
            freq_dict[id] = list(set(matches))

    saveCSV(freq_dict, "freq_pref_20230703.csv")
    return freq_dict
            

# qualifications = 7, preferred = 8, skills = 10

## 각 채용링크의 skills 속 skill set 정리하기

In [73]:
import csv

def skills():
    freq_dict = dict()

    with open("../data/crawled_data_combined.csv", 'r', encoding="utf-8") as f:
        skill_reader = csv.reader(f)
        skill = list(skill_reader)

        for row in skill[1:]:
            id = row[0]
            freq_dict[id] = row[10]

    print(len(freq_dict))

    saveCSV(freq_dict, "freq_skill_20230703.csv")

# id = 0,  skills = 10



In [74]:
skills()

5450
Dictionary keys and values saved to CSV file successfully.


### 각 리스트에 데이터를 넣어서 리스트=컬럼으로 한 csv파일 작성, 합쳐진 csv파일 작성

In [ ]:
import ast

total_dict = dict()

ids = []
with open("freq_skill_20230703.csv","r",encoding="949") as skill:
    skill_reader = csv.reader(skill)
    skill_rows = list(skill_reader)
    skills = []

with open("freq_qual_20230703.csv","r",encoding="949") as qual:
    qual_reader = csv.reader(qual)
    qual_rows = list(qual_reader)
    quals = []

with open("freq_pref_20230703.csv","r",encoding="949") as pref:
    pref_reader = csv.reader(pref)
    pref_rows = list(pref_reader)
    prefs = []

    print(len(skill_rows), len(qual_rows), len(pref_rows))
    count = 0
    for row in range(1, len(skill_rows)):
        if (skill_rows[row][0] == qual_rows[row][0] == pref_rows[row][0]):
            id = skill_rows[row][0]
            skill = skill_rows[row][10]
            qual = qual_rows[row][10]
            pref = pref_rows[row][10]

            # 각 리스트에도 넣어준다.
            ids.append(id)
            skills.append(skill)
            quals.append(qual)
            prefs.append(pref)
            
            # string 안 list를 list로 변환 empty string이면 []로 바꿔준다.
            # skills_L = ast.literal_eval(skill) if skill else []
            # quals_L = ast.literal_eval(qual) if qual else []
            # prefs_L = ast.literal_eval(pref) if pref else []

            # combined_L = skills_L + quals_L + prefs_L

            # total_dict[id] = combined_L

        if row % 500 == 0:
            print(skill_rows[row][0], qual_rows[row][0], pref_rows[row][0])
            print(skill_rows[row][10], qual_rows[row][10], pref_rows[row][10])
            

In [ ]:
import pandas as pd

# Create the DataFrame
df = pd.DataFrame({'id': skill_keys, 'qualification': qual_values, 'preffered': pref_values, 'skill': skill_values})

# Save the DataFrame as a CSV file with column names
df.to_csv('qual_pref_skill_count.csv', index=False, encoding="utf-8")